Fully Homomorphically Encrypted Fashion-MNIST CNN Example
=========================================================

- This example will download Fashion-MNIST (a drop in replacement for MNIST)
- Prepare and encrypt Fashion-MNIST
- Train a very basic CNN on Fashion-MNIST
- Output some classification of the Fashion-MNIST testing set and calculate its error

Download Fashion-MNIST
----------------------

- Get Fashion-MNIST as a zipped up set of CSVs
- Unizp Fashion-MNIST

In [ ]:
import os
import requests
import zipfile

In [ ]:
cwd = os.getcwd() # current working directory
print(cwd)

In [ ]:
data_dir = os.path.join(cwd, "datasets")
if os.path.exists(data_dir):
    pass
else:
    os.mkdir(data_dir)
print(data_dir)

In [ ]:
mnist_zip = os.path.join(data_dir, "mnist.zip")
if os.path.exists(mnist_zip):
    print("Skipping mnist download")
else:
    print("Downloading Fashion-MNIST")
    mnist_url = "http://nextcloud.deepcypher.me/s/wjLa6YFw8Bcbra9/download"
    r = requests.get(mnist_url, allow_redirects=True, verify=False)
    with open(mnist_zip, "wb") as f:
        f.write(r.content)

In [ ]:
unzip_dir = os.path.join(data_dir, "mnist")
if os.path.exists(unzip_dir):
    pass
else:
    os.mkdir(unzip_dir)
with zipfile.ZipFile(mnist_zip, "r") as zip_ref:
    zip_ref.extractall(unzip_dir)

"Wrangle"/ prepare Fashion-MNIST
--------------------------------

- Read in the Fashion-MNIST CSVs
- Split training and testing features (x) from target (y)
- Normalise x and y (in the range 0-1 to prevent infinite numbers when using our approximations)

In [ ]:
import pandas as pd
import numpy as np
import tqdm

In [ ]:
train_file = os.path.join(unzip_dir, "fashion-mnist_train.csv") 
test_file = os.path.join(unzip_dir, "fashion-mnist_test.csv")
train = pd.read_csv(train_file)
test = pd.read_csv(train_file)
# train

In [ ]:
train_y = train.iloc[:, 0]
train_x = train.iloc[:, 1:]
test_x = train.iloc[:, 1:]
test_y = test.iloc[:, 0]
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()
print(train_x.shape)
print(train_y.shape)

Define Neural Network
---------------------

- Use [Networkx](https://networkx.org/) to construct a **multi-directed-graph** as a neural network
- Nodes for this graph are abstractions of neural network components with forward, backward (backpropogation), update (weight update/ optimisation), and costs (computational depth of traversal to the node)
- We use Nodes that inherit from the abstract base class [fhez.nn.graph.node.Node](https://python-fhez.readthedocs.io/en/latest/nodes/node.html#node) so if you need to define your own type of node inherit from this to match the API the network traverser expects

In [ ]:
import networkx as nx
from fhez.nn.graph.io import IO
from fhez.nn.operations.cc import CC # Cross Correlation
from fhez.nn.operations.sum import Sum
from fhez.nn.activation.relu import RELU # Rectified Linear Unit (approximation)
from fhez.nn.layer.ann import ANN # Dense/ Artificial Neural Network
from fhez.nn.activation.softmax import Softmax
from fhez.nn.loss.cce import CCE # categorical cross entropy

from fhez.nn.operations.encrypt import Encrypt
from fhez.nn.operations.decrypt import Decrypt
from fhez.nn.operations.enqueue import Enqueue
from fhez.nn.operations.dequeue import Dequeue

In [ ]:
graph = nx.MultiDiGraph()
classes = np.array([0,1,2,3,4,5,6,7,8,9])

# add nodes to graph with names (for easy human referencing), and objects for what those nodes are

# CONSTRUCT CNN 
# with intermediary decrypted sum to save on some complexity later
graph.add_node("CNN-products", node=CC(weights=(1,6,6), stride=[1,4,4], bias=0))
graph.add_node("CNN-dequeue", node=Dequeue())
graph.add_edge("CNN-products", "CNN-dequeue")
graph.add_node("CNN-sum-of-products", node=Sum())
graph.add_edge("CNN-dequeue", "CNN-sum-of-products")
graph.add_node("CNN-enqueue", node=Enqueue())
graph.add_edge("CNN-sum-of-products", "CNN-enqueue")
graph.add_node("CNN-activation", node=RELU())
graph.add_edge("CNN-enqueue", "CNN-activation")

# CONSTRUCT DENSE FOR EACH CLASS
# we want to get the network to regress some prediction one for each class
graph.add_node("Dense-enqueue", node=Enqueue())
for i in classes:
    graph.add_node("Dense-{}".format(i), node=ANN())
    graph.add_edge("CNN-activation", "Dense-{}".format(i))
    graph.add_node("Dense-activation-{}".format(i), node=RELU())
    graph.add_edge("Dense-{}".format(i), "Dense-activation-{}".format(i))
    graph.add_edge("Dense-activation-{}".format(i), "Dense-enqueue")
    
# CONSTRUCT CLASSIFIER
# we want to turn the dense outputs into classification probabilities using softmax and how wrong/ right we are using Categorical Cross-Entropy (CCE) as our loss function
graph.add_node("Softmax", node=Softmax())
graph.add_edge("Dense-enqueue", "Softmax")
graph.add_node("Loss-CCE", node=CCE())
graph.add_edge("Softmax", "Loss-CCE")
print(graph)

Visualise the graph
-------------------

Visualise the graph using networkx inbuilt plots like spring. This will always be avaliable to you but is fairly rudamentary.

In [ ]:
nx.draw(graph, with_labels=True)

Visualise the graph in a much nicer way using graphviz

In [ ]:
# import sys
# !{sys.executable} -m pip install pyvis

In [ ]:
# from pyvis.network import Network
# net = Network(notebook=True)
# net.from_nx(graph)
# net.show("graph.html")

Parameterise Encoding/ Encryption and Create an Encrypted Generator
-------------------------------------------------------------------

- Import our encryption library
- Parameterise the encryption tailored to the computations we will use
- Create a generator that returns encrypted versions of whatever we give it row-by-row (since each image is encoded as a row here)

In [ ]:
import seal # https://github.com/Huelse/SEAL-Python OR https://github.com/DreamingRaven/python-seal
from fhe.nn.layer.cnn import Layer_CNN # from this library
from fhe.nn.layer.ann import Layer_ANN # from this library
from fhe.rearray import ReArray # meta encryption object from this library

In [ ]:
encryption_parameters = {
            "scheme": seal.scheme_type.CKKS,
            "poly_modulus_degree": 8192*2,
            "coefficient_modulus":
                [45, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 45],
            "scale": pow(2.0, 30),
            "cache": True,
}

In [ ]:
# Generate Encrypted data peace-meal (as it can get very large)
def row_encrypted_generator(data: np.ndarray, shape: tuple):
    """Generate encrypted data of desired shape from rows."""
    for row in data:
        row = np.reshape(row, newshape=shape) / 255 # reshape to image shape and normalise between 0-1
        yield ReArray(row, **encryption_parameters)

Train Using Encrypted Data
--------------------------

- Instantiate our neural networks
- Call our encrypted data generator to generate data as needed
- Compute the forward pass of our neural networks
- Compute the backward pass of our neural networks

In [ ]:
cnn = Layer_CNN(weights=( 1, 6, 6 ), stride=[ 1, 4, 4 ], bias=0)
dense = None
for cyphertext in row_encrypted_generator(data=train_x, shape=( 1, 28, 28 )):
    cnn_acti = cnn.forward(cyphertext)
    if dense is None:
        dense = Layer_ANN(weights=(len(cnn_acti),), bias=0)
    dense.forward(cnn_acti)

Test Using Encrypted Data
-------------------------